In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

**`Load Data`**
+ `one-hot` format

In [2]:
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
print('X_train.shape :', mnist.train.images.shape)
print('Y_train.shape :', mnist.train.labels.shape)

X_train.shape : (55000, 784)
Y_train.shape : (55000, 10)


In [4]:
batch_size = 100
n_batch = mnist.train.num_examples // batch_size  # 整除

In [5]:
def variable_summaries(var):
    with tf.name_scope('Summary'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)

In [6]:
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, [None, 784], name='x-input')
    y = tf.placeholder(tf.float32, [None, 10],  name='y-input')

In [7]:
with tf.name_scope('keep_prob'):
    keep_prob = tf.placeholder(tf.float32)

In [8]:
with tf.name_scope('layer_1'):
    with tf.name_scope('weights'):
        Weight_L1 = tf.Variable(tf.truncated_normal([784,300], stddev=0.1), name='w')  # (input, output)
        variable_summaries(Weight_L1)
    with tf.name_scope('bias'):
        Biases_L1 = tf.Variable(tf.random_normal([1, 300]), name='bias')
        variable_summaries(Biases_L1)
    with tf.name_scope('z'):
        Z_L1 = tf.matmul(x, Weight_L1) + Biases_L1
    with tf.name_scope('tanh'):
        A_L1 = tf.nn.tanh(Z_L1)
    with tf.name_scope('Dropout'):
        Drop_L1 = tf.nn.dropout(A_L1, keep_prob)

In [9]:
with tf.name_scope('layer_2'):
    with tf.name_scope('weights'):
        Weight_L2 = tf.Variable(tf.truncated_normal([300,100], stddev=0.1), name='w')  # (input, output)
        variable_summaries(Weight_L2)
    with tf.name_scope('bias'):
        Biases_L2 = tf.Variable(tf.random_normal([1, 100]), name='bias')
        variable_summaries(Biases_L2)
    with tf.name_scope('z'):
        Z_L2 = tf.matmul(Drop_L1, Weight_L2) + Biases_L2
    with tf.name_scope('tanh'):
        A_L2 = tf.nn.tanh(Z_L2)
    with tf.name_scope('Dropout'):
        Drop_L2 = tf.nn.dropout(A_L2, keep_prob)

In [10]:
with tf.name_scope('layer_3'):
    with tf.name_scope('weights'):
        Weight_L3 = tf.Variable(tf.truncated_normal([100,10], stddev=0.1), name='w')  # (input, output)
        variable_summaries(Weight_L1)
    with tf.name_scope('bias'):
        Biases_L3 = tf.Variable(tf.random_normal([1, 10]), name='bias')
        variable_summaries(Biases_L3)
    with tf.name_scope('z'):
        Z_L3 = tf.matmul(Drop_L2, Weight_L3) + Biases_L3
    with tf.name_scope('softmax'):
        Prediction = tf.nn.softmax(Z_L3)

In [11]:
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=Prediction))
    tf.summary.scalar('loss', loss)

In [12]:
lr = tf.Variable(0.001) # 1e-3

with tf.name_scope('train'):
    optimizer = tf.train.AdamOptimizer(lr)
    train = optimizer.minimize(loss)

In [13]:
with tf.name_scope('accuary'):
    with tf.name_scope('correct_prediction'):
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(Prediction, 1))
        # return a bool-list
    with tf.name_scope('accuary'):
        accuary = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        tf.summary.scalar('accuary', accuary)

In [14]:
# 合并所有的监测目标
merged = tf.summary.merge_all()

In [15]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    writer = tf.summary.FileWriter('TensorBoard/Class_05/logs_01/', sess.graph)
    
    for epoch in range(60):
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            summary, _ = sess.run([merged, train], feed_dict={
                x : batch_xs,
                y : batch_ys,
                keep_prob : 0.87
            })
        
        writer.add_summary(summary, epoch)
        
        test_acc = sess.run(accuary, feed_dict={
            x:mnist.test.images,
            y:mnist.test.labels,
            keep_prob : 1.0
        })
        train_acc = sess.run(accuary, feed_dict={
            x:mnist.train.images,
            y:mnist.train.labels,
            keep_prob : 1.0
        })

        print("Iter " + str(epoch) + "  Test Acuuary " + str(test_acc) + "  Train Acuuary " + str(train_acc))

Iter 0  Test Acuuary 0.9408  Train Acuuary 0.94241816
Iter 1  Test Acuuary 0.9518  Train Acuuary 0.9564546
Iter 2  Test Acuuary 0.9602  Train Acuuary 0.9665091
Iter 3  Test Acuuary 0.9654  Train Acuuary 0.9715273
Iter 4  Test Acuuary 0.9659  Train Acuuary 0.9739818
Iter 5  Test Acuuary 0.9644  Train Acuuary 0.97670907
Iter 6  Test Acuuary 0.971  Train Acuuary 0.98087275
Iter 7  Test Acuuary 0.9718  Train Acuuary 0.98309094
Iter 8  Test Acuuary 0.9722  Train Acuuary 0.98432726
Iter 9  Test Acuuary 0.9711  Train Acuuary 0.9850364
Iter 10  Test Acuuary 0.9734  Train Acuuary 0.9866545
Iter 11  Test Acuuary 0.9738  Train Acuuary 0.98785454
Iter 12  Test Acuuary 0.974  Train Acuuary 0.9873091
Iter 13  Test Acuuary 0.9761  Train Acuuary 0.9892727
Iter 14  Test Acuuary 0.9746  Train Acuuary 0.98956364
Iter 15  Test Acuuary 0.9751  Train Acuuary 0.98892725
Iter 16  Test Acuuary 0.9766  Train Acuuary 0.99023634
Iter 17  Test Acuuary 0.9755  Train Acuuary 0.9905273
Iter 18  Test Acuuary 0.9766  T